# SIADEX HTN ENGINE

imports requireds

In [1]:
!pip install ..
!pip install ../unified-planning

Processing /workspaces/up-siadex
  Preparing metadata (setup.py) ... done
  Created wheel for up-siadex: filename=up_siadex-0.0.1-py3-none-any.whl size=8662148 sha256=bfe45b2eb139b9f678605d6e81e7d55764ea205839b86a2296e5bb0b7192037d
  Stored in directory: /home/codespace/.cache/pip/wheels/63/55/a1/26a2eb3a61793baa41bdcf5daf2fa5975374c48f8639a70b08
Successfully built up-siadex
  Attempting uninstall: up-siadex
    Found existing installation: up-siadex 0.0.1
    Uninstalling up-siadex-0.0.1:
      Successfully uninstalled up-siadex-0.0.1
Processing /workspaces/up-siadex/unified-planning
  Preparing metadata (setup.py) ... done
  Created wheel for unified-planning: filename=unified_planning-0.4.2-py3-none-any.whl size=352562 sha256=b5cdf430d535a5881c9b4a3da83ba0f0a4e910640f5335c742527be8e52418a2
  Stored in directory: /home/codespace/.cache/pip/wheels/c4/05/f2/821c8767d2971ed17287b1008afb0607f41dad844571a5b40f
Successfully built unified-planning
  Attempting uninstall: unified-planning
  

In [2]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


## Registering the engine

In order to use `SIADEX`, we need to register it among the set of planning engines available for the UP library as follows.

In [3]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

## Reading a problem from a file

In [4]:
reader = PDDLReader()
problem = reader.parse_problem("../unified-planning/unified_planning/test/pddl/htn-transport/domain.hddl", "../unified-planning/unified_planning/test/pddl/htn-transport/problem.hddl")

## Solving the problem with siadex

In [5]:
with env.factory.OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        for i,a in enumerate(result.plan.actions):
            print(f"{i}: {a}")
    else:
        print('No plan found!')

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_env().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_11950/3527194102.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

SIADEX found a valid plan!
The plan is: 

0: drive(truck-0, city-loc-2, city-loc-1)
1: pick-up(truck-0, city-loc-1, package-0, capacity-0, capacity-1)
2: drive(truck-0, city-loc-1, city-loc-0)
3: drop(truck-0, city-loc-0, package-0, capacity-0, capacity-1)
4: drive(truck-0, city-loc-0, city-loc-1)
5: pick-up(truck-0, city-loc-1, package-1, capacity-0, capacity-1)
6: drive(truck-0, city-loc-1, city-loc-0)
7: drop(truck-0, city-loc-0, package-1, capacity-0, capacity-1)


/home/codespace/.python/current/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7f7d283a4b80> cannot solve this kind of problem!
  warn(msg)


In [6]:
def solve_with_siadex(problem):
    with env.factory.OneshotPlanner(name='siadex') as p:
        result = p.solve(problem)
        print(result.log_messages)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{p.name()} found a valid plan!')
            print(f'The plan is: \n')
            for i,a in enumerate(result.plan.actions):
                print(f"{i}: {a}")
            return result.plan
        else:
            return []
            print('No plan found!')

In [28]:
htn = HierarchicalProblem()

Location = UserType("Location")

l1 = htn.add_object("l1", Location)
l2 = htn.add_object("l2", Location)
l3 = htn.add_object("l3", Location)
l4 = htn.add_object("l4", Location)

loc = Fluent("is_on", l=Location)
htn.add_fluent(loc, default_initial_value=False)


connected = Fluent("connected", l1=Location, l2=Location)
htn.add_fluent(connected, default_initial_value=False)
htn.set_initial_value(connected(l1, l2), True)
htn.set_initial_value(connected(l2, l3), True)
htn.set_initial_value(connected(l3, l4), True)
htn.set_initial_value(connected(l4, l3), True)
htn.set_initial_value(connected(l3, l2), True)
htn.set_initial_value(connected(l2, l1), True)
htn.set_initial_value(loc(l1), True)



move = InstantaneousAction("move", l_from=Location, l_to=Location)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
move.add_precondition(loc(l_from))
move.add_precondition(connected(l_from, l_to))
move.add_effect(loc(l_from), False)
move.add_effect(loc(l_to), True)
htn.add_action(move)

go = htn.add_task("go", target=Location)

go_noop = Method("go-noop", target=Location)
go_noop.set_task(go)
target = go_noop.parameter("target")
go_noop.add_precondition(loc(target))

htn.add_method(go_noop)

go_recursive = Method(
    "go-recursive", source=Location, inter=Location, target=Location
)

go_recursive.set_task(go, go_recursive.parameter("target"))
source = go_recursive.parameter("source")
inter = go_recursive.parameter("inter")
target = go_recursive.parameter("target")

go_recursive.add_precondition(loc(source))
go_recursive.add_precondition(connected(source, inter))

t1 = go_recursive.add_subtask(move, source, inter)
t2 = go_recursive.add_subtask(go, target)
go_recursive.set_ordered(t1, t2)
htn.add_method(go_recursive)

go1 = htn.task_network.add_subtask(go, l4)


In [29]:
solve_with_siadex(htn)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_11950/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move(l1, l2)
1: move(l2, l3)
2: move(l3, l4)


[move(l1, l2), move(l2, l3), move(l3, l4)]

In [49]:
reader = HPDLReader()
miconic = reader.parse_problem("../examples/ipc/Miconic/domain.hpdl", "../examples/ipc/Miconic/problem.hpdl")
# rover = reader.parse_problem("../examples/ipc/Rover/domain.hpdl","../examples/ipc/Rover/problem.hpdl")
# satellite = reader.parse_problem("../examples/ipc/Satellite/domain.hpdl","../examples/ipc/Satellite/problem.hpdl")
# smartphone = reader.parse_problem("../examples/ipc/SmartPhone/domain.hpdl","../examples/ipc/SmartPhone/problem.hpdl")
# transport = reader.parse_problem("../examples/ipc/Transport/domain.hpdl","../examples/ipc/Transport/problem.hpdl")
# translog = reader.parse_problem("../examples/ipc/UM-Translog/domain.hpdl","../examples/ipc/UM-Translog/problem.hpdl")
# zeno = reader.parse_problem("../examples/ipc/Zenotravel/domain.hpdl","../examples/ipc/Zenotravel/problem.hpdl")

In [46]:
reader_hddl = PDDLReader()
rover = reader_hddl.parse_problem("../examples/ipc/Rover/domain.hddl","../examples/ipc/Rover/problem.hddl")

 

ParseException: Expected ')', found '('  (at char 2796), (line:77, col:9)

In [11]:
writer = HPDLWriter(satellite)

writer.write_domain("../test/satellite.hpdl")
writer.write_problem("../test/satellite-problem.hpdl")


In [48]:
solve_with_siadex(zeno)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_11950/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



/home/codespace/.python/current/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7f7d283a4b80> cannot solve this kind of problem!
  warn(msg)


[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]


[]

In [50]:
solve_with_siadex(miconic)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_11950/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



/home/codespace/.python/current/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7f7d283a4b80> cannot solve this kind of problem!
  warn(msg)


[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move_primitive(f0, f0)
1: board_primitive(p0, f0)
2: move_primitive(f0, f1)
3: debark_primitive(p0, f1)
4: move_primitive(f1, f3)
5: board_primitive(p2, f3)
6: move_primitive(f3, f1)
7: debark_primitive(p2, f1)
8: move_primitive(f1, f2)
9: board_primitive(p4, f2)
10: move_primitive(f2, f1)
11: debark_primitive(p4, f1)
12: move_primitive(f1, f0)
13: board_primitive(p1, f0)
14: move_primitive(f0, f3)
15: debark_primitive(p1, f3)
16: move_primitive(f3, f3)
17: board_primitive(p3, f3)
18: move_primitive(f3, f2)
19: debark_primitive(p3, f2)


[move_primitive(f0, f0), board_primitive(p0, f0), move_primitive(f0, f1), debark_primitive(p0, f1), move_primitive(f1, f3), board_primitive(p2, f3), move_primitive(f3, f1), debark_primitive(p2, f1), move_primitive(f1, f2), board_primitive(p4, f2), move_primitive(f2, f1), debark_primitive(p4, f1), move_primitive(f1, f0), board_primitive(p1, f0), move_primitive(f0, f3), debark_primitive(p1, f3), move_primitive(f3, f3), board_primitive(p3, f3), move_primitive(f3, f2), debark_primitive(p3, f2)]